In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

# Load training split
dataset = load_dataset("openpecha/OCR-Google_Books")

# Example features
print(dataset)
# {
#     'filename': 'image_001.jpg',
#     'label': '༄༅། །རྒྱལ་པོ་ ལ་ གཏམ་ གྱི་ བྱུང་ བ་ བཤད་ པ།',
#     'url': 'https://...',
#     'BDORC_work_id': 'W...',
#     'char_len': 25,
#     'script': 'ScriptTibt',
#     'print_method': 'Modern'
# }

In [ ]:
dataset['train'][0]

## Remove Unused Columns

In [ ]:
#dataset = dataset.remove_columns(['BDRC_work_id', 'char_len', 'script', 'print_method']).rename_column('filename', 'id')
#dataset['train'][0]

## Add Placeholder Image Column

In [ ]:
"""from datasets import Image as ImageFeature

def add_placeholder_image(example):
    #Add a None placeholder for the image column

    example['image'] = None
    return example

# Add the image column with placeholders
print("Adding placeholder image column...")
dataset = dataset.map(
    add_placeholder_image,
    desc="Adding image column"
)

dataset = dataset.cast_column('image', ImageFeature())

dataset['train'][0]"""

## Download and Append Images

In [12]:
import requests
from PIL import Image
import io
from datasets import Dataset
import pandas as pd

def download_image(example):
    """
    Function to be used with dataset.map() to download images for each example
    """
    try:
        # Download the image
        response = requests.get(example['url'])
        response.raise_for_status()
        
        # Open image and convert to PIL Image object
        image = Image.open(io.BytesIO(response.content))
        example['image'] = image
        
    except Exception as e:
        # If download fails, set image to None and continue
        print(f"Failed to download image from {example.get('url', 'unknown')}. Error: {e}")
        example['image'] = None
    
    return example

In [13]:
eval = dataset['eval'].map(download_image)

Map:   0%|          | 0/75136 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
process_split_in_chunks('test', chunk_size=5000, start_chunk=0)

In [ ]:
process_split_in_chunks('train', chunk_size=5000, start_chunk=0)

In [ ]:
process_split_in_chunks('train', chunk_size=5000, start_chunk=21)

In [ ]:
#dataset.push_to_hub("openpecha/OCR-Google_Books")